<a href="https://colab.research.google.com/github/ahmarhafeez1/object_detection_yolov5/blob/main/Object_Detection_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Set up the environment
!pip install torch torchvision
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

In [ ]:
import torch
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time

In [ ]:
# Step 3: Load the YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

# Set the classes we're interested in (person, car, and animal classes from COCO)
model.classes = [0, 2, 3, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]

# Set confidence threshold
model.conf = 0.25  # NMS confidence threshold
model.iou = 0.45   # NMS IoU threshold


In [ ]:
def getFrame(sec, quality=0.8):
    # JavaScript to capture frame from webcam
    js = Javascript('''
        async function getFrame(sec, quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();
            await new Promise(r => setTimeout(r, sec * 1000));
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            const result = canvas.toDataURL('image/jpeg', quality);
            stream.getVideoTracks()[0].stop();
            div.remove();
            return result;
        }
    ''')
    display(js)
    data = eval_js('getFrame({}, {})'.format(sec, quality))
    binary = b64decode(data.split(',')[1])
    return PIL.Image.open(io.BytesIO(binary))

In [ ]:
def processFrame(frame):
    # Convert PIL Image to OpenCV format
    open_cv_image = np.array(frame.convert('RGB'))
    open_cv_image = open_cv_image[:, :, ::-1].copy()

    # Perform detection
    results = model(open_cv_image)

    # Process results
    for det in results.xyxy[0]:  # det is (x1, y1, x2, y2, conf, cls)
        x1, y1, x2, y2, conf, cls = det.tolist()
        label = model.names[int(cls)]
        color = (0, 255, 0) if label == 'person' else (0, 0, 255) if label == 'car' else (255, 0, 0)
        cv2.rectangle(open_cv_image, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
        cv2.putText(open_cv_image, f'{label} {conf:.2f}', (int(x1), int(y1) - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Convert back to PIL Image
    return PIL.Image.fromarray(open_cv_image[:, :, ::-1])


In [ ]:
# Step 5: Main loop for capturing and processing frames
def runDetection(duration=30, interval=1):
    start_time = time.time()
    while time.time() - start_time < duration:
        frame = getFrame(interval)
        processed_frame = processFrame(frame)
        display(processed_frame)

# Step 6: Run the detection
runDetection(duration=30, interval=1)

In [ ]:
from google.colab import files
import io
import matplotlib.pyplot as plt
import torch
import cv2
import numpy as np
from PIL import Image

def upload_and_detect():
    # Upload an image
    uploaded = files.upload()

    # Get the uploaded image
    for filename in uploaded.keys():
        # Read the image file
        image = Image.open(io.BytesIO(uploaded[filename]))

        # Convert PIL Image to OpenCV format
        open_cv_image = np.array(image.convert('RGB'))
        open_cv_image = open_cv_image[:, :, ::-1].copy()

        # Perform detection
        results = model(open_cv_image)

        # Get the rendered image with detections from YOLOv5
        rendered_image = results.render()[0]

        # Convert back to RGB for displaying
        rgb_image = cv2.cvtColor(rendered_image, cv2.COLOR_BGR2RGB)

        # Display the result
        plt.figure(figsize=(12, 8))
        plt.imshow(rgb_image)
        plt.axis('off')
        plt.title('Detected Objects')
        plt.show()

        # Print detection summary
        print(results.pandas().xyxy[0])

# Run the function to upload and detect
upload_and_detect()